<a href="https://colab.research.google.com/github/Allzer/Machine_learning/blob/main/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BA_%D0%B1%D0%BB%D0%B8%D0%B6%D0%BD%D0%B8%D1%85_%D0%BF%D0%B5%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У меня не получилось делать так, как в примере, я буду делать по своему, а потом сравню результаты


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy.io as sio
from sklearn.model_selection import train_test_split #разбивает данные для тестирования и тренировки

#библиотеки курса
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#загружаем тестовые данные
def load_data_mat(filename, max_samples, seed=64):
    raw = sio.loadmat(filename) #загружаем файл
    x = raw['X'] #Передаём массивы принадлежащие Х
    y = raw['y'] #Передаём массивы принадлежащие у

    x = np.moveaxis(x, [3], [0]) #Перемещение оси массива. 3 ось мы ставим на нулевую
    y = y.flatten() #Возвращает одномерный массив (для уменьшения массива до одного измерения)

    y[y == 10] = 0 #все значения 10 присваивается знаение 0

    np.random.seed(seed)
    samples = np.random.choice(np.arange(x.shape[0]), max_samples, replace=False) #x.shape[0] передаёт в данном случае
    #рандомную строку 2D массива
    #np.choice - вернет max_sample случайных чисeл от 0 до x.shape[0].
    return x[samples].astype(np.float32), y[samples] #возвращаем массив иксов и игриков с одинаковыми семплами

def load_svhn(folder, max_train, max_test):

    #train_x - массив (num_train,32,32,3) - тренировочные изображения
    #train_y - массив (num_train) - training labels - тренировочный разделитель
    #test_X, np array (num_test, 32, 32, 3) - тестовые изображения
    #test_y, np array of int (num_test) - тестовый разделитель

    train_X, train_y = load_data_mat(os.path.join(folder, "train_32x32.mat"), max_train) #Загружает данные для тренировки
    test_X, test_y = load_data_mat(os.path.join(folder, "test_32x32.mat"), max_test) #Загружает данные для тестов
    return train_X, train_y, test_X, test_y #Возвращает загруенные данные

def random_split_train_val(X, y, num_val, seed=35):
    '''
    Randomly splits dataset into training and validation

    Arguments:
    X - np-массив с выборками
    y - np-массив с метками
    num_val - количество образцов для проверки
    seed - random seed

    Returns:
    train_X, np array (num_train, 32, 32, 3) - обучающие изображения
    train_y, np array of int (num_train) - обучающие метки
    val_X, np array (num_val, 32, 32, 3) - validation images
    val_y, np array of int (num_val) - validation labels
    '''
    np.random.seed(seed)

    indices = np.arange(X.shape[0]) #передаёт в переменную одномерны массив
    np.random.shuffle(indices) #измените последовательность на месте, перетасовав ее содержимое.

    train_indices = indices[:-num_val] #получаем все элементы массива кроме элемента с индексом -num_val
    train_X = X[train_indices]
    train_y = y[train_indices]

    val_indices = indices[-num_val:]
    val_X = X[val_indices]
    val_y = y[val_indices]

    return train_X, train_y, val_X, val_y

train_x, train_y, test_x, test_y = load_svhn("drive/MyDrive/data", max_train=1000, max_test=1000)

Загружаем класс KNN

In [ ]:
from urllib.parse import parse_qs
class KNN:
    """
    K-neariest-neighbor classifier using L1 loss
    """
    def __init__(self, k=1):
        self.k = k

    def fit(self, X, y):
        self.train_X = X
        self.train_y = y

    def predict(self, X, num_loops=1):
        '''
        Uses the KNN model to predict clases for the data samples provided

        Arguments:
        X, np array (num_samples, num_features) - samples to run
           through the model
        num_loops, int - which implementation to use

        Returns:
        predictions, np array of ints (num_samples) - predicted class
           for each sample
        '''
        if num_loops == 0:
            dists = self.compute_distances_no_loops(X)
        elif num_loops == 1:
            dists = self.compute_distances_one_loop(X)
        else:
            dists = self.compute_distances_two_loops(X)

        if self.train_y.dtype == np.bool:
            return self.predict_labels_binary(dists)
        else:
            return self.predict_labels_multiclass(dists)


    def compute_distances_two_loops(self, X):
      num_train = self.train_X.shape[0]
      num_test = X.shape[0]
      dists = np.zeros((num_test, num_train),np.float32)

      for i_test in range(num_test):
        for i_train in range(num_train):
          dists[i_test][i_train] = np.sum(np.abs((X[i_test] - self.train_X[i_train])))
          pass
      return  dists
    #(X[i_test] - self.train_X[i_train]) - вектор рсстояния(см в тетради)
    #X - переданный тестовый набор
    #self.train_X - хранятся проверочные картинки

    def compute_distances_one_loop(self, X):
        '''
        Computes L1 distance from every sample of X to every training sample
        Vectorizes some of the calculations, so only 1 loop is used

        Arguments:
        X, np array (num_test_samples, num_features) - samples to run

        Returns:
        dists, np array (num_test_samples, num_train_samples) - array
           with distances between each test and each train sample
        '''
        num_train = self.train_X.shape[0]
        num_test = X.shape[0]
        dists = np.zeros((num_test, num_train), np.float32)
        for i_test in range(num_test):
            dists[i_test] = np.sum(np.abs(X[i_test] - self.train_X), axis=1)
        return dists
    #В данном случае dists - не просто сумма, а сумма по строкам записанная в строки


    def compute_distances_no_loops(self, X):
      num_train = self.train_X.shape[0]
      num_test = X.shape[0]
      dists = np.zeros((num_test, num_train), np.float32)

      dists = np.sum(np.abs(X[None,:]-self.train_X[:,None]),2)
      return dists.T
      pass



    def predict_labels_binary(self, dists):
        '''
        Returns model predictions for binary classification case

        Arguments:
        dists, np array (num_test_samples, num_train_samples) - array
           with distances between each test and each train sample

        Returns:
        pred, np array of bool (num_test_samples) - binary predictions
           for every test sample
        '''
        num_test = dists.shape[0]
        pred = np.zeros(num_test, np.bool)
        for i in range(num_test):
            index = dists[i].argsort()[:self.k] #вектор индексов; i - конкретная тестовая картинка. argsort() - выдаёт k ближайших индексов
            pred[i] = bool(np.median(self.train_y[index]))
            pass
        return pred

    def predict_labels_multiclass(self, dists):
        '''
        Returns model predictions for multi-class classification case

        Arguments:
        dists, np array (num_test_samples, num_train_samples) - array
           with distances between each test and each train sample

        Returns:
        pred, np array of int (num_test_samples) - predicted class index
           for every test sample
        '''
        num_test = dists.shape[0]
        num_test = dists.shape[0]
        pred = np.zeros(num_test, np.int)
        for i in range(num_test):
            # TODO: Implement choosing best class based on k
            # nearest training samples
            closest_y = self.train_y[np.argsort(dists[i, :])[:self.k]] #получаем индексы наименьших k-дистанций
            # sort by value then get indexes by slicing till k (first k in each row)
            #print((closest_y))
            u, indices = np.unique(closest_y, return_inverse=True)
            #print('U',u, indices)
            #print(u[np.argmax(np.bincount(indices))])
            pred[i] = u[np.argmax(np.bincount(indices))]
        return pred



In [ ]:
def binary_classification_metrics(prediction, ground_truth):
    '''
    Computes metrics for binary classification

    Arguments:
    prediction, np array of bool (num_samples) - model predictions
    ground_truth, np array of bool (num_samples) - true labels

    Returns:
    precision, recall, f1, accuracy - classification metrics
    '''
    tp = np.sum(np.logical_and(prediction, ground_truth))
    fp = np.sum(np.greater(prediction, ground_truth))
    fn = np.sum(np.less(prediction, ground_truth))
    precision = tp/(tp + fp)
    recall = tp/(tp+fn)

    accuracy = np.sum(prediction==ground_truth)/prediction.size
    f1 = precision*recall/(precision+recall)

    return precision, recall, f1, accuracy


def multiclass_accuracy(prediction, ground_truth):
    '''
    Computes metrics for multiclass classification

    Arguments:
    prediction, np array of int (num_samples) - model predictions
    ground_truth, np array of int (num_samples) - true labels

    Returns:
    accuracy - ratio of accurate predictions to total samples
    '''
    # TODO: Implement computing accuracy

    return accuracy_score(ground_truth, prediction)

In [ ]:
sample = 5
plot_index = 1

for exaample_idx in range(sample):
  for class_idx in range(10):
    plt.subplot(5,10, plot_index)
    image = train_x[train_y == class_idx][exaample_idx]
    plt.imshow(image.astype(np.uint8))
    plt.axis('off')
    plot_index +=1
plt.show()

Начнём с бинарной классификации


In [ ]:
binary_train_mask = (train_y == 0) | (train_y == 9)
binary_train_X = train_x[binary_train_mask] #закидываем массивы в которых лежат ячейки с фотографиями 0 и 9
binary_train_y = train_y[binary_train_mask] == 0 #закидываем значения, которые = 0. Если 0 - True, если 9 - False

binary_test_mask = (test_y == 0) | (test_y == 9)
binary_test_X = test_x[binary_test_mask]
binary_test_y = test_y[binary_test_mask] == 0

binary_train_X = binary_train_X.reshape(binary_train_X.shape[0], -1)
binary_test_X = binary_test_X.reshape(binary_test_X.shape[0], -1)

In [ ]:
binary_train_X.shape

Вычисляет расстояние L1 от каждой выборки X до каждой обучающей выборки.
Использует простейшую реализацию с 2 циклами

Аргументы:
X, np_array (num_test_samples, num_features) - образцы для запуска

Возвращается:
dists, np-массив (num_test_samples, num_train_samples) - массив
с указанием расстояний между каждым тестом и каждой последовательной выборкой

Сделаем только с одним циклом, потому что остальное  у меня не работает...

In [ ]:
knn_classifier = KNN(k=1)
knn_classifier.fit(binary_train_X, binary_train_y)

In [ ]:
dists = knn_classifier.compute_distances_two_loops(binary_test_X)
assert np.isclose(dists[0, 10], np.sum(np.abs(binary_test_X[0] - binary_train_X[10])))
#dists - матрица всех расстояний

dists2 = knn_classifier.compute_distances_one_loop(binary_test_X)
assert np.isclose(dists2[0, 10], np.sum(np.abs(binary_test_X[0] - binary_train_X[10])))

dists3 = knn_classifier.compute_distances_no_loops(binary_test_X)
assert np.isclose(dists3[0, 10], np.sum(np.abs(binary_test_X[0] - binary_train_X[10])))

In [ ]:
prediction = knn_classifier.predict(binary_test_X)
(prediction==binary_test_y) #True - good, False - no good

In [ ]:
%timeit knn_classifier.compute_distances_two_loops(binary_test_X)
%timeit knn_classifier.compute_distances_one_loop(binary_test_X)
%timeit knn_classifier.compute_distances_no_loops(binary_test_X)

Metrics.py

In [ ]:
precision, recall, f1, accuracy = binary_classification_metrics(prediction, binary_test_y)
print("KNN with k = %s" % knn_classifier.k)
print("Accuracy: %4.2f, Precision: %4.2f, Recall: %4.2f, F1: %4.2f" % (accuracy, precision, recall, f1))

KNN with k = 1
Accuracy: 0.55, Precision: 0.57, Recall: 0.57, F1: 0.28


In [ ]:
knn_classifier_3 = KNN(k=6) #На параметре k = 6 - модель работает лучше. Понял эмперически
knn_classifier_3.fit(binary_train_X, binary_train_y)
prediction = knn_classifier_3.predict(binary_test_X)

precision, recall, f1, accuracy = binary_classification_metrics(prediction, binary_test_y)
print("KNN with k = %s" % knn_classifier_3.k)
print("Accuracy: %4.2f, Precision: %4.2f, Recall: %4.2f, F1: %4.2f" % (accuracy, precision, recall, f1))

Кросс-валидация. Первый вариант

In [ ]:
num_folds = 5
train_folds_X = []
train_folds_y = []

train_folds_X = np.array_split(binary_train_X[:(binary_train_X.shape[0]//num_folds)*num_folds], num_folds)
train_folds_y = np.array_split(binary_train_y[:(binary_train_X.shape[0]//num_folds)*num_folds], num_folds)

knn_classifier = KNN(k=3)
knn_classifier.k = 1
'''
Наш датасет не делится ровно на 5 частей. поэтому код выдаёт ошибку. Чтобы её избежать я использовал операцию целочисленного деления.
Мол, если мы делим на 5, а элементов 24, то благодаря данному способу мы в папки закинем не 5 элемментов, а 4
'''
# ЗАДАЧА: разделите тренировочные данные на 5 частей и сохраните их в train_folds_X/train_folds_you
k_choices = [1, 2, 3, 5, 8, 10, 15, 20, 25, 50]
k_to_f1 = {}  # dict mapping k values to mean F1 scores (int -> float)

 # ЗАДАЧА: выполнить перекрестную проверку
 # Просмотрите каждую складку и используйте ее для тестирования, а все остальные складки - для тренировки
 # Выполните обучение и создайте показатель оценки F1 на основе набора проверочных данных
 # Усредните значение F1 по всем сгибам и запишите его в k_to_f1

for k in k_choices:
  knn_classifier.k = k
  f1_tot = 0
  for numfold in range(num_folds):
    Xtrain = np.concatenate(train_folds_X[:numfold] + train_folds_X[numfold+1:])
    Ytrain = np.concatenate(train_folds_y[:numfold] + train_folds_y[numfold+1:])
    knn_classifier.fit(Xtrain, Ytrain)

    prediction = knn_classifier.predict(train_folds_X[numfold], num_loops=1    )
    gt = train_folds_y[numfold]

    precision, recall, f1, acuuracy = binary_classification_metrics(prediction, gt)
    f1_tot += f1
    pass
  k_to_f1[k] = f1_tot/num_folds
  pass

for k in sorted(k_to_f1):
  print('k = %d, f1 = %f' % (k,k_to_f1[k]))

Кросс-валидация
Попробуем найти лучшее значение параметра k для алгоритма KNN!

Для этого мы воспользуемся k-fold cross-validation (https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation). Мы разделим тренировочные данные на 5 фолдов (folds), и по очереди будем использовать каждый из них в качестве проверочных данных (validation data), а остальные -- в качестве тренировочных (training data).

В качестве финальной оценки эффективности k мы усредним значения F1 score на всех фолдах. После этого мы просто выберем значение k с лучшим значением метрики.

Кросс-валидация. Второй вариант - лучше работает


In [ ]:
num_folds = 5
train_folds_X = np.array(np.array_split(binary_train_X, num_folds))
train_folds_y = np.array(np.array_split(binary_train_y, num_folds))

# ЗАДАЧА: разделите тренировочные данные на 5 частей и сохраните их в train_folds_X/train_folds_you
k_choices = [1, 2, 3, 5, 8, 10, 15, 20, 25, 50]
k_to_f1 = {}  # dict mapping k values to mean F1 scores (int -> float)

 # ЗАДАЧА: выполнить перекрестную проверку
 # Просмотрите каждую складку и используйте ее для тестирования, а все остальные складки - для тренировки
 # Выполните обучение и создайте показатель оценки F1 на основе набора проверочных данных
 # Усредните значение F1 по всем сгибам и запишите его в k_to_f1
for k in k_choices:
    f1_arr = []
    for i in range(num_folds):
        knn_classifier = KNN(k=k)
        knn_classifier.fit(np.concatenate(np.delete(train_folds_X, i, axis=0)), np.concatenate(np.delete(train_folds_y, i, axis=0)))
        prediction = knn_classifier.predict(train_folds_X[i])
        accuracy, precision, recall, f1 = binary_classification_metrics(prediction, train_folds_y[i])
        f1_arr.append(f1)
    k_to_f1[k] = np.average(f1_arr)

for k in sorted(k_to_f1):
    print('k = %d, f1 = %f' % (k, k_to_f1[k]))

Проверим, как хорошо работает лучшее значение k на тестовых данных (test data)

In [ ]:
# TODO Установите наилучшее значение k на наилучшее значение, найденное путем перекрестной проверки

best_k = 3

best_knn_classifier = KNN(k=best_k)
best_knn_classifier.fit(binary_train_X, binary_train_y)
prediction = best_knn_classifier.predict(binary_test_X)

precision, recall, f1, accuracy = binary_classification_metrics(prediction, binary_test_y)
print("Best KNN with k = %s" % best_k)
print("Accuracy: %4.2f, Precision: %4.2f, Recall: %4.2f, F1: %4.2f" % (accuracy, precision, recall, f1))

Многоклассовая классификация (multi-class classification)

In [ ]:
# Now let's use all 10 classes
train_X = train_x.reshape(train_x.shape[0], -1)
test_X = test_x.reshape(test_x.shape[0], -1)

knn_classifier = KNN(k=1)
knn_classifier.fit(train_X, train_y)

In [ ]:
# ЗАДАЧА: Реализовать predict_labels_multiclass
predict = knn_classifier.predict(test_X)

# ЗАДАЧА: Реализовать мультиклассовую точность
accuracy = multiclass_accuracy(predict, test_y)
print("Accuracy: %4.2f" % accuracy)

Снова кросс-валидация. Теперь нашей основной метрикой стала точность (accuracy), и ее мы тоже будем усреднять по всем фолдам.



In [ ]:
# Find the best k using cross-validation based on accuracy
num_folds = 5
train_folds_X = np.array(np.array_split(train_X, num_folds))
train_folds_y = np.array(np.array_split(train_y, num_folds))

#ЗАДАЧА: разделите тренировочные данные на 5 частей и сохраните их в train_folds_X/train_folds_you

k_choices = [1, 2, 3, 5, 8, 10, 15, 20, 25, 50]
k_to_accuracy = {}

for k in k_choices:
    f1_arr = []
    for i in range(num_folds):
        knn_classifier = KNN(k=k)
        knn_classifier.fit(np.concatenate(np.delete(train_folds_X, i, axis=0)),np.concatenate(np.delete(train_folds_y, i, axis=0)))
        prediction = knn_classifier.predict(train_folds_X[i])
        accuracy, precision, recall, f1 = binary_classification_metrics(prediction, train_folds_y[i])
        f1_arr.append(f1)
    k_to_accuracy[k] = np.average(f1_arr)

for k in sorted(k_to_accuracy):
    print('k = %d, f1 = %f' % (k, k_to_accuracy[k]))

Финальный тест - классификация на 10 классов на тестовой выборке (test data)

In [ ]:
# TODO Set the best k as a best from computed
best_k = 8

best_knn_classifier = KNN(k=best_k)
best_knn_classifier.fit(train_X, train_y)
prediction = best_knn_classifier.predict(test_X)

# Accuracy should be around 20%!
accuracy = multiclass_accuracy(prediction, test_y)
print("Accuracy: %4.2f" % accuracy)